In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
import cv2

from utils import BackprojectDepth, Project3D

In [37]:
H, W = 384, 640
T_left = torch.from_numpy(np.array([[ 0.6712, -0.0146,  0.7412,  0.0952],
                                    [ 0.0148,  0.9999,  0.0062,  0.0170],
                                    [-0.7412,  0.0068,  0.6713, -0.1428],
                                    [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=np.float32)).unsqueeze(0)
T_right = torch.from_numpy(np.array([[ 0.5453,  0.0140, -0.8382, -0.4028],
                                     [-0.0013,  0.9999,  0.0158, -0.0318],
                                     [ 0.8383, -0.0076,  0.5452, -0.5721],
                                     [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=np.float32)).unsqueeze(0)
K_center = torch.from_numpy(np.array([[721.1671,   0.0000, 306.7841,   0.0000],
                     [  0.0000, 688.9274, 194.5127,   0.0000],
                     [  0.0000,   0.0000,   1.0000,   0.0000],
                     [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=np.float32)).unsqueeze(0)
K_left = torch.from_numpy(np.array([[349.4441,   0.0000, 318.9036,   0.0000],
                     [  0.0000, 333.4656, 185.8930,   0.0000],
                     [  0.0000,   0.0000,   1.0000,   0.0000],
                     [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=np.float32)).unsqueeze(0)
K_right = torch.from_numpy(np.array([[350.6630,   0.0000, 312.9119,   0.0000],
                     [  0.0000, 334.5016, 193.0759,   0.0000],
                     [  0.0000,   0.0000,   1.0000,   0.0000],
                     [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=np.float32)).unsqueeze(0)
D = torch.from_numpy(np.load("data/depth.npy"))
img_left = cv2.cvtColor(cv2.imread("data/CAM_LEFT.jpg"), cv2.COLOR_BGR2RGB)
img_right = cv2.cvtColor(cv2.imread("data/CAM_RIGHT.jpg"), cv2.COLOR_BGR2RGB)
transform = transforms.ToTensor()
img_left = transform(img_left).unsqueeze(0)
img_right = transform(img_right).unsqueeze(0)

In [38]:
def reconstruct_image(src, depth, T, K_src, K_tar):
    # initialize projecting functions
    backproject_depth = BackprojectDepth(1, H, W)
    project_3d = Project3D(1, H, W)

    # do reconstruction 
    cam_points = backproject_depth(depth, torch.inverse(K_tar))
    pix_coords = project_3d(cam_points, K_src, T)
    img_cam1_warpped =  F.grid_sample(src, pix_coords, \
        padding_mode="zeros", align_corners=True)
    
    return img_cam1_warpped

In [45]:
rect_from_left = reconstruct_image(img_left, D, \
                T_left, K_left, K_center)
rect_from_right = reconstruct_image(img_right, D, \
                T_right, K_right, K_center)


In [46]:
rect_both = rect_from_left + rect_from_right
save_image(rect_both, "data/OVL.png")